In [2]:
%autosave 0
# %matplotlib inline
%matplotlib notebook

from __future__ import division, print_function
import numpy as np
from tensorflow.python.client import device_lib
from keras import backend as K
import tensorflow as tf
import keras
from keras.datasets import cifar10


print(device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(keras.__version__)

Autosave disabled
[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8922926907652105629
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11332668621
locality {
  bus_id: 1
}
incarnation: 11553048389818214341
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:06:00.0"
, name: "/gpu:1"
device_type: "GPU"
memory_limit: 11330614068
locality {
  bus_id: 1
}
incarnation: 6262823173465881690
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:07:00.0"
]
2.0.5


In [24]:
from keras.layers import Input, Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Add, Activation
from keras.models import Model, load_model
from keras import optimizers
# from keras.utils import multi_gpu_model

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
input_train_shape = x_train.shape[1:]

n_out = 10
n_fc1 = 256

ff_model = load_model('cifar_feedforward_180711.h5');
conv1_ffw = ff_model.get_layer('conv_1')
conv2_ffw = ff_model.get_layer('conv_2')
conv3_ffw = ff_model.get_layer('conv_3')
conv4_ffw = ff_model.get_layer('conv_4')
fc5_ffw = ff_model.get_layer('fc_5')
fc6_ffw = ff_model.get_layer('fc_6')
fc7_ffw = ff_model.get_layer('fc_7')

input_layer = Input(shape=(input_train_shape), name='input')
conv1_ff1 = Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff1', trainable=False, weights=conv1_ffw.get_weights())(input_layer)
conv1_rc1 = Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_rc1', trainable=True)(conv1_ff1)
conv1_mrg1 = Add()([conv1_ff1, conv1_rc1])
bn1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_1')(conv1_mrg1)
pool1 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_1')(bn1)
conv2_ff1 = Conv2D(128, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff2', trainable=False, weights=conv2_ffw.get_weights())(pool1)
conv2_rc1 = Conv2D(128, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_rc2', trainable=True)(conv2_ff1)
conv2_mrg1 = Add()([conv2_ff1, conv2_rc1])
bn2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_2')(conv2_mrg1)
pool2 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_2')(bn2)
conv3_ff1 = Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff3', trainable=False, weights=conv3_ffw.get_weights())(pool2)
conv3_rc1 = Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_rc3', trainable=True)(conv3_ff1)
conv3_mrg1 = Add()([conv3_ff1, conv3_rc1])
bn3 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_3')(conv3_mrg1)
pool3 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool3')(bn3)
conv4_ff1 = Conv2D(1024, (4,4), strides=(1,1), padding='valid', activation='relu', name='conv_ff4', trainable=False, weights=conv4_ffw.get_weights())(pool3)
conv4_rc1 = Conv2D(1024, (1,1), strides=(1,1), padding='valid', activation='relu', name='conv_rc4', trainable=True)(conv4_ff1)
conv4_mrg1 = Add()([conv4_ff1, conv4_rc1])
bn4 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_4')(conv4_mrg1)
flat1 = Flatten(name='flat_1')(bn4)
fc5_ff1 = Dense(1024, activation='relu', name='fc_ff5', trainable=False, weights=fc5_ffw.get_weights())(flat1)
fc5_rc1 = Dense(1024, activation='relu', name='fc_rc5', trainable=True)(fc5_ff1)
fc5_mrg1 = Add()([fc5_ff1, fc5_rc1])
bn5 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_5')(fc5_mrg1)
fc6_ff1 = Dense(256, activation='relu', name='fc_ff6', trainable=False, weights=fc6_ffw.get_weights())(bn5)
fc6_rc1 = Dense(256, activation='relu', name='fc_rc6', trainable=True)(fc6_ff1)
fc6_mrg1 = Add()([fc6_ff1, fc6_rc1])
bn6 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_6')(fc6_mrg1)
fc7_ff1 = Dense(10, activation='linear', name='fc_ff7', trainable=False, weights=fc7_ffw.get_weights())(bn6)
fc7_rc1 = Dense(10, activation='linear', name='fc_rc7', trainable=True)(fc7_ff1)
fc7_mrg1 = Add()([fc7_ff1, fc7_rc1])
output_layer = Activation('softmax')(fc7_mrg1)




In [25]:
sgd = optimizers.SGD(lr=2, decay=1e-6, momentum=0.9)
model = Model(input_layer, output_layer)
# pmodel = multi_gpu_model(model)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv_ff1 (Conv2D)                (None, 32, 32, 64)    1792        input[0][0]                      
____________________________________________________________________________________________________
conv_rc1 (Conv2D)                (None, 32, 32, 64)    36928       conv_ff1[0][0]                   
____________________________________________________________________________________________________
add_33 (Add)                     (None, 32, 32, 64)    0           conv_ff1[0][0]                   
                                                                   conv_rc1[0][0]          

In [26]:
print (x_train.shape)
print (y_train.shape)

# scale intensities between 0 and 1
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print('min:', x_train[0].min())
print('max:', x_train[0].max())


(50000, 32, 32, 3)
(50000, 1)
min: 0.0
max: 1.0


In [27]:
n_epochs = 5
n_batch_size = 4
labels_train = keras.utils.to_categorical(y_train, n_out)
labels_test = keras.utils.to_categorical(y_test, n_out)


history = model.fit(x_train, labels_train,
                    epochs=n_epochs,
                    batch_size=n_batch_size)

evaluation = model.evaluate(x_test, labels_test, verbose=0)

print('\n[Train parameters]')
for item in history.params:
    print(item+':', history.params[item])
    
print('\n[Model evaluation]')
print('test', history.params['metrics'][0], format(np.mean(evaluation[0]), '.4f'))
print('test', history.params['metrics'][1], format(np.mean(evaluation[1]), '.4f'))

Epoch 1/5
50000/50000 [==============================] - 185s - loss: 1.1581 - acc: 0.6117   
Epoch 2/5
50000/50000 [==============================] - 183s - loss: 0.9922 - acc: 0.6668   
Epoch 3/5
50000/50000 [==============================] - 183s - loss: 0.9512 - acc: 0.6789   
Epoch 4/5
50000/50000 [==============================] - 183s - loss: 0.9213 - acc: 0.6922   
Epoch 5/5
50000/50000 [==============================] - 184s - loss: 0.8884 - acc: 0.7036   

[Train parameters]
metrics: ['loss', 'acc']
samples: 50000
batch_size: 4
epochs: 5
do_validation: False
verbose: 1

[Model evaluation]
test loss 0.8388
test acc 0.7265


In [28]:
model.save('cifar_recurrentcomplex_180712.h5')

In [7]:
# Playground
conv1_ffw.set_weights(1)

TypeError: object of type 'int' has no len()